In [1]:
using StatisticalRethinking, CmdStan, StanMCMCChain
gr(size=(500,500));

ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

df2 = filter(row -> row[:age] >= 18, df)
mean_height = mean(df2[:height])
df2[:height_c] = convert(Vector{Float64}, df2[:height]) .- mean_height
first(df2, 5)

max_height_c = maximum(df2[:height_c])
min_height_c = minimum(df2[:height_c])

heightsmodel = "
// Inferring a Rate
data {
  int N;
  real h[N];
}
parameters {
  real<lower=0> sigma;
  real<lower=$(min_height_c),upper=$(max_height_c)> mu;
}
model {
  // Priors for mu and sigma
  mu ~ normal(178, 20);
  sigma ~ uniform( 0 , 50 );

  // Observed heights
  h ~ normal(mu, sigma);
}
";

stanmodel = Stanmodel(name="heights", monitors = ["mu", "sigma"],model=heightsmodel,
  output_format=:mcmcchain);

heightsdata = Dict("N" => length(df2[:height]), "h" => df2[:height_c]);

rc, chn, cnames = stan(stanmodel, heightsdata, ProjDir, diagnostics=false,
  CmdStanDir=CMDSTAN_HOME);

describe(chn)

serialize("m4.2s.jls", chn)

chn2 = deserialize("m4.2s.jls")

describe(chn2)


File /Users/rob/.julia/dev/StatisticalRethinking/scripts/04/tmp/heights.stan will be updated.


Inference for Stan model: heights_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.031, 0.031, 0.031, 0.034) seconds, 0.13 seconds total
Sampling took (0.046, 0.068, 0.060, 0.041) seconds, 0.22 seconds total

                 Mean     MCSE  StdDev     5%    50%   95%    N_Eff  N_Eff/s    R_hat
lp__             -932  2.4e-02     1.0   -934   -932  -931  1.9e+03  8.6e+03  1.0e+00
accept_stat__    0.91  5.1e-03    0.11   0.67   0.95   1.0  4.8e+02  2.2e+03  1.0e+00
stepsize__       0.90  6.2e-02   0.088   0.82   0.92   1.0  2.0e+00  9.3e+00  6.9e+13
treedepth__       2.0  1.5e-01    0.83    1.0    2.0   4.0  2.9e+01  1.4e+02  1.1e+00
n_leapfrog__      5.6  1.5e+00     8.9    1.0    3.0    23  3.4e+01  1.6e+02  1.0e+00
divergent__      0.00      nan    0.00   0.00   0.00  0.00      nan      nan      nan
energy__      

end of m4.2s#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*